In [1]:

# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
credentials_db2 = {
    'username': 'xxxxx',
    'password': """xxxxx""",
    'database': 'BLUDB',
    'host': 'dashdb-xxxxx.services.eu-gb.bluemix.net',
    'port': '50000',
}
quandl_api_key = "xxxxx"


Quandl Python Module | Quandl
https://www.quandl.com/tools/python

In [2]:
!pip install quandl


In [23]:
#quandlから日経平均の過去5日分を取得
import quandl
import pandas as pd
import  matplotlib.pyplot as plt

quandl.ApiConfig.api_key = quandl_api_key
df = quandl.get("CHRIS/CME_NK2", rows=5)


In [24]:
df.tail(10)

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,
2020-06-10,22945.0,23215.0,22780.0,22895.0,50.0,22890.0,4283.0,11747.0
2020-06-11,22895.0,22960.0,21645.0,21785.0,1165.0,21725.0,11512.0,12528.0
2020-06-12,NaN,22375.0,NaN,22320.0,550.0,22110.0,0.0,0.0
2020-06-15,NaN,NaN,21935.0,21935.0,245.0,21865.0,0.0,0.0
2020-06-16,NaN,22505.0,NaN,22420.0,435.0,22300.0,0.0,0.0


In [25]:
#RDBに書き出すために列名を大文字化して、空白を変換
df1=df.reset_index()
df1.rename(columns=lambda s:s.upper().replace(' ', '_'), inplace=True)
df1

,DATE,OPEN,HIGH,LOW,LAST,CHANGE,SETTLE,VOLUME,PREVIOUS_DAY_OPEN_INTEREST
0,2020-06-10,22945.0,23215.0,22780.0,22895.0,50.0,22890.0,4283.0,11747.0
1,2020-06-11,22895.0,22960.0,21645.0,21785.0,1165.0,21725.0,11512.0,12528.0
2,2020-06-12,NaN,22375.0,NaN,22320.0,550.0,22110.0,0.0,0.0
3,2020-06-15,NaN,NaN,21935.0,21935.0,245.0,21865.0,0.0,0.0
4,2020-06-16,NaN,22505.0,NaN,22420.0,435.0,22300.0,0.0,0.0


Watson StudioのJupyter NotebookからDb2の表にデータ出力をする - Qiita
https://qiita.com/tetsuya1969/items/867ae78132e969fd078a

In [26]:
#DB接続
from ibmdbpy import IdaDataBase, IdaDataFrame

dsn = 'DASHDB;Database={};Hostname={};Port={};PROTOCOL=TCPIP;UID={};PWD={}'.format(
    credentials_db2['database'],
    credentials_db2['host'],
    credentials_db2['port'],
    credentials_db2['username'],
    credentials_db2['password']
)
from ibmdbpy import IdaDataBase, IdaDataFrame

idadb = IdaDataBase(dsn)



In [27]:
#初回投入時にはテーブルを作成する
#ida_df=idadb.as_idadataframe(df1, 'CME_NK2', clear_existing = True, primary_key='DATE')
#idadb.commit

In [28]:
#重なるデータをテーブルから削除
dates=df.index

In [29]:
idadb.ida_query('DELETE FROM CME_NK2 WHERE DATE BETWEEN \'{}\' AND \'{}\''.format(dates.min(),dates.max()))

In [30]:
#取得したデータをINSERT
ida_df = IdaDataFrame(idadb, 'CME_NK2')
idadb.append(ida_df, df1)
idadb.commit

Uploading 5 rows (maxnrow was set to 888)


<bound method IdaDataBase.commit of <ibmdbpy.base.IdaDataBase object at 0x7f39659506a0>>

In [31]:
#ida_df.sort(columns='DATE',inplace=True)
ida_df.tail(10)

,DATE,OPEN,HIGH,LOW,LAST,CHANGE,SETTLE,VOLUME,PREVIOUS_DAY_OPEN_INTEREST
0,2020-06-03 00:00:00,22625.0,22975.0,22510.0,22940.0,265.0,22940.0,128.0,85.0
1,2020-06-05 00:00:00,22790.0,23365.0,22590.0,23240.0,460.0,23205.0,8168.0,183.0
2,2020-06-11 00:00:00,22895.0,22960.0,21645.0,21785.0,1165.0,21725.0,11512.0,12528.0
3,2020-06-04 00:00:00,22940.0,23000.0,22535.0,22770.0,195.0,22745.0,160.0,124.0
4,2020-06-10 00:00:00,22945.0,23215.0,22780.0,22895.0,50.0,22890.0,4283.0,11747.0
5,2020-06-08 00:00:00,23260.0,23335.0,23005.0,23260.0,55.0,23260.0,8162.0,4700.0
6,2020-06-09 00:00:00,23275.0,23275.0,22755.0,22925.0,320.0,22940.0,7147.0,7992.0
7,2020-06-12 00:00:00,NaN,22375.0,NaN,22320.0,550.0,22110.0,0.0,0.0
8,2020-06-15 00:00:00,NaN,NaN,21935.0,21935.0,245.0,21865.0,0.0,0.0
9,2020-06-16 00:00:00,NaN,22505.0,NaN,22420.0,435.0,22300.0,0.0,0.0


In [32]:
idadb.close()

Connection closed.
